In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

In [3]:
spark = SparkSession.builder \
    .appName("ETL") \
    .getOrCreate()

24/08/24 22:34:19 WARN Utils: Your hostname, basel resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlp3s0)
24/08/24 22:34:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/24 22:34:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data1 = spark.read.csv('../../data/raw/data2.csv',header=True, inferSchema=True)
data2= spark.read.csv('../../data/raw/data3.csv',header=True,inferSchema=True)

In [16]:
## flatten and grouping the first data
column_df=data1.select('skills')
rdd = column_df.rdd.map(lambda row: row[0])
splitted_rdd = rdd.flatMap(lambda line : line.split('·'))
word_pairs1 = splitted_rdd.map(lambda word : (word.strip(),1))
word_counted= word_pairs1.reduceByKey(lambda a ,b : a+b)


423

In [10]:
df2= data2.groupBy("Skill").agg(sum("Count").alias("Total_count"))
word_pairs2 = df2.rdd.map(lambda row: f"({row['Skill']}: {row['Total_count']})")


['(creativity: 1)',
 '(marketplace: 1)',
 '(develop applications: 2)',
 '(scale: 1)',
 '(athena: 1)',
 '(management: 6)',
 '(metadata: 2)',
 '(database engineer: 2)',
 '(data integrity: 2)',
 '(nosql: 2)',
 '(disabilities: 2)',
 '(data visualization: 4)',
 '(finance: 2)',
 '(data infrastructure: 2)',
 '(lina: 1)',
 '(etap: 2)',
 '(warehousing: 3)',
 '(design build: 2)',
 '(scientists healthcare: 4)',
 '(drawing: 2)',
 '(informatica: 1)',
 '(research: 5)',
 '(web services: 2)',
 '(forth: 1)',
 '(design information: 8)',
 '(big data: 2)',
 '(excellent business: 4)',
 '(authorization: 2)',
 '(leadership: 3)',
 '(infrastructure: 19)',
 '(data warehousing: 2)',
 '(backends: 1)',
 '(present design: 4)',
 '(autocad: 2)',
 '(read: 1)',
 '(catalogues: 1)',
 '(inform model: 2)',
 '(energy: 2)',
 '(tableau: 1)',
 '(data modelling data: 3)',
 '(ophthalmology: 1)',
 '(data science: 2)',
 '(troubleshooting: 2)',
 '(rehabilitation: 1)',
 '(integration: 1)',
 '(kinesis aws: 4)',
 '(data mart: 2)',
 '(

In [22]:
union_rdd= word_pairs1.union(word_counted)
union_rdd.save('../../data/transformed/data_final2')

In [29]:
union_rdd_df = union_rdd.toDF(['skill','count'])
union_rdd_df.collect()
union_rdd_df.coalesce(1).write.csv('../../data/transformed/data_final',header=True)